# Librerías

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# https://www.ambientebogota.gov.co/estaciones-rmcab

# Limpieza

## Cargar datos

In [77]:
years = range(2000, 2025, 1)
dfs = pd.DataFrame()
for year in years:
    df = pd.read_excel(f"../Datos/RMCAB/Raw_RMCAB_{year}.xlsx")
    dfs = pd.concat([dfs, df])

# Limpieza inicial
dfs = dfs.drop_duplicates()
dfs = dfs.drop(columns=["Unnamed: 0","Máx","Mín"])

In [52]:
#dfs = ex.copy()

## Limpiar datos

In [80]:
dfs.head()

,parámetros,Unidades,01,02,03,04,05,06,07,08,...,17,18,19,20,21,22,23,24,Fecha,Estación
0,PM10,µg/m3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000-01-01,Guaymaral
1,Vel Viento 10M,m/s,"0,4",0,0,0,0,0,0,1,...,"2,2",2,1,"0,8","0,4","0,1","0,1",0,2000-01-01,Guaymaral
2,Dir Viento 10M,Grados,217,196,11,0,189,349,357,7,...,275,267,230,188,180,199,195,194,2000-01-01,Guaymaral
3,Temperatura,°C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000-01-01,Guaymaral
4,Temperatura 8M,°C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000-01-01,Guaymaral


In [86]:
ex = dfs.copy()

### Datos panel

In [120]:
panel_raw = pd.melt(dfs, id_vars=['parámetros', 'Fecha', 'Estación',"Unidades"], 
                    var_name='Hora', value_name='Valor')
panel_raw["Valor"] = panel_raw["Valor"].str.replace(",",".").astype(float)

#panel_raw["Par_Und"] = panel_raw["parámetros"] + " (" + panel_raw["Unidades"] + ")"
#panel_raw.to_csv(f'../Datos/RMCAB/Raw/Panel_RMCAB.csv', index=False)
panel_raw["Hora"] = panel_raw["Hora"].astype(str)
panel_raw["Hora"] = panel_raw["Hora"] + ":00:00"
panel_raw["Hora"] = panel_raw["Hora"].replace("24:00:00", "00:00:00")
panel_raw["Hora"] = pd.to_datetime(panel_raw["Hora"], format='%H:%M:%S').dt.time
panel_raw["Fecha_Hora"] = pd.to_datetime(panel_raw["Fecha"].astype(str) + " " + panel_raw["Hora"].astype(str))

In [123]:
panel_raw.to_csv(f'../Datos/RMCAB/Raw/Panel_RMCAB.csv', index=False)

In [124]:
panel_raw

,parámetros,Fecha,Estación,Unidades,Hora,Valor,Fecha_Hora
0,PM10,2000-01-01,Guaymaral,µg/m3,01:00:00,NaN,2000-01-01 01:00:00
1,Vel Viento 10M,2000-01-01,Guaymaral,m/s,01:00:00,0.4,2000-01-01 01:00:00
2,Dir Viento 10M,2000-01-01,Guaymaral,Grados,01:00:00,217.0,2000-01-01 01:00:00
3,Temperatura,2000-01-01,Guaymaral,°C,01:00:00,NaN,2000-01-01 01:00:00
4,Temperatura 8M,2000-01-01,Guaymaral,°C,01:00:00,NaN,2000-01-01 01:00:00
...,...,...,...,...,...,...,...
36213283,Presion Baro,2024-09-13,Puente Aranda,mmHg,00:00:00,565.0,2024-09-13 00:00:00
36213284,Rad Solar,2024-09-13,Puente Aranda,W/M²,00:00:00,0.0,2024-09-13 00:00:00
36213285,HR,2024-09-13,Puente Aranda,%,00:00:00,56.0,2024-09-13 00:00:00
36213286,BBP,2024-09-13,Puente Aranda,%,00:00:00,7.3,2024-09-13 00:00:00


#### Info estaciones

In [144]:
estaciones = panel_raw["Estación"].unique().tolist()

resultados = []

for estacion in estaciones:
    df_estacion = panel_raw[panel_raw["Estación"] == estacion]
    params = df_estacion["parámetros"].unique().tolist()
    
    for param in params:
        try:
            min_date = df_estacion[df_estacion["parámetros"] == param]["Fecha_Hora"].min()
            max_date = df_estacion[df_estacion["parámetros"] == param]["Fecha_Hora"].max()
            len_df = len(df_estacion[df_estacion["parámetros"] == param])
            nulls = df_estacion[df_estacion["parámetros"] == param]["Valor"].isnull().sum()
            nulls_percentage = nulls / len_df * 100

            # Acumular los resultados en la lista
            resultados.append({
                "Estación": estacion,
                "Parámetro": param,
                "Fecha Inicio": min_date,
                "Fecha Fin": max_date,
                "Total_Observaciones": len_df,
                "Total_Nulos": nulls,
                "% Disponiblidad de datos": 100 - nulls_percentage
            })
        except Exception as e:
            print(f"Error en {param}: {e}")
            continue

# Convertir la lista de resultados en un DataFrame
info_estaciones = pd.DataFrame(resultados)

In [151]:
estaciones = info_estaciones["Estación"].unique().tolist()

with pd.ExcelWriter("../Datos/RMCAB/Info_Estaciones.xlsx") as writer:
    for estacion in estaciones:
        df = info_estaciones[info_estaciones["Estación"] == estacion]
        df.to_excel(writer, sheet_name=f'{estacion}', index=False)

In [177]:
fecha_inicio_por_estacion = info_estaciones.groupby('Estación')['Fecha Inicio'].min()
fecha_fin_por_estacion = info_estaciones.groupby('Estación')['Fecha Fin'].max()

# Combinar los resultados en un DataFrame
fechas_por_estacion = pd.DataFrame({
    'Fecha_Inicio': fecha_inicio_por_estacion,
    'Fecha_Fin': fecha_fin_por_estacion
})

# Resetear el índice para que 'Estación' sea una columna
fechas_por_estacion.reset_index(inplace=True)


In [173]:
#info_estaciones.to_excel("../Datos/RMCAB/Info_Estaciones_DF_completo.xlsx", index=False)

In [ ]:
import seaborn as sns

plt.figure(figsize=(8, 120))
ax = sns.barplot(data=info_estaciones, x='% Disponiblidad de datos', y='Parámetro', hue='Estación')

plt.title('Disponibilidad de datos por parámetro por estación')
plt.xlabel('Disponibilidad de datos (%)')
plt.ylabel('Parámetro')

bars = ax.containers

for container in bars:
    ax.bar_label(container, fmt='%.2f%%', label_type='edge') 

plt.legend(title='Estación', bbox_to_anchor=(1.05, 1), loc='upper left')

plt.show()

#### Diccionario datos panel

In [ ]:
panel_raw[["parámetros","Unidades"]].drop_duplicates()

In [191]:
panel_raw["Unidades"].unique()

array(['µg/m3', 'm/s', 'Grados', '°C', '%', 'mmHg', 'W/M²', 'mm', 'ppb',
       'ppm', 'ng/m3'], dtype=object)

In [197]:
diccionario_panel = {
    "Hora": {
        "Descripción": "Hora de la medición. Formato: HH:MM:SS, 24 horas",
        "Tipo": "Object",
        "Valores posibles": "Formato HH:MM:SS"
    },
    "Fecha": {
        "Descripción": "Fecha de la medición. Formato: AAAA-MM-DD. Desde 2000-01-01 hasta 2024-09-13",
        "Tipo": "Date",
        "Valores posibles": "Fechas en el rango 2000-01-01 a 2024-09-13"
    },
    "Fecha_Hora": {
        "Descripción": "Fecha y hora de la medición. Formato: AAAA-MM-DD HH:MM:SS",
        "Tipo": "DateTime",
        "Valores posibles": "Fechas y horas en el rango 2000-01-01 01:00:00 a 2024-09-13 23:00:00"
    },
    "Estacion": {
        "Descripción": "Estación meteorológica donde se realizó la medición",
        "Tipo": "Object",
        "Valores posibles": """'Guaymaral', 'MinAmbiente', 'Suba', 'Usaquen',
        'Carvajal - Sevillana', 'Puente Aranda', 'Las Ferias',
        'Centro de Alto Rendimiento', 'Kennedy', 'Tunal', 'San Cristobal',
        'Movil 7ma', 'Bolivia', 'Fontibon', 'Usme', 'Jazmin',
        'Ciudad Bolivar', 'Colina', 'Movil Fontibon'"""
    },
    "parámetros": {

        "Descripción": "Variables del clima de Bogotá",
        "Tipo": "Object",
        "Valores posibles": """'PM10', 'Vel Viento 10M', 'Dir Viento 10M', 'Temperatura',
        'Temperatura 8M', 'Temperatura 20M', 'HR', 'Presion Baro',
        'Rad Solar', 'Precipitacion', 'OZONO', 'NO', 'NO2', 'NOX', 'PM2.5',
        'CO', 'SO2', 'Vel Viento', 'Dir Viento', 'OZONO Envea', 'BBP',
        'BC ug/m3', 'Tempe Inter', 'Humedad Inter', 'SO2 Envea', 'Temp_4m',
        'UV-BC', 'HR_2m'""" 
    },
    "Unidades": {
        "Descripción": "Unidad de medida de cada variable",
        "Tipo": "Object",
        "Valores posibles": """'µg/m3', 'm/s', 'Grados', '°C', '%', 'mmHg', 'W/M²', 'mm', 'ppb',
        'ppm', 'ng/m3'""" 
    }

    }

df_diccionario = pd.DataFrame.from_dict(diccionario_panel, orient="index").reset_index()
df_diccionario.columns = ["Variable","Descripcción","Tipo","Valores posibles"]

df_diccionario.to_excel("../Datos/RMCAB/Diccionario_Panel_RMCAB.xlsx", index=False)


In [199]:
# Definir el diccionario con descripciones
descriptions = {
    'PM10': 'Partículas en suspensión de diámetro inferior a 10 micrómetros (µg/m3)',
    'Vel Viento 10M': 'Velocidad del viento a 10 metros de altura (m/s)',
    'Dir Viento 10M': 'Dirección del viento a 10 metros de altura (Grados)',
    'Temperatura': 'Temperatura ambiente (°C)',
    'Temperatura 8M': 'Temperatura a 8 metros de altura (°C)',
    'Temperatura 20M': 'Temperatura a 20 metros de altura (°C)',
    'HR': 'Humedad relativa (%)',
    'Presion Baro': 'Presión barométrica (mmHg)',
    'Rad Solar': 'Radiación solar (W/M²)',
    'Precipitacion': 'Precipitación acumulada (mm)',
    'OZONO': 'Concentración de ozono (ppb)',
    'NO': 'Concentración de monóxido de nitrógeno (ppb)',
    'NO2': 'Concentración de dióxido de nitrógeno (ppb)',
    'NOX': 'Concentración total de óxidos de nitrógeno (ppb)',
    'PM2.5': 'Partículas en suspensión de diámetro inferior a 2.5 micrómetros (µg/m3)',
    'CO': 'Concentración de monóxido de carbono (ppm)',
    'SO2': 'Concentración de dióxido de azufre (ppb)',
    'Vel Viento': 'Velocidad del viento (m/s)',
    'Dir Viento': 'Dirección del viento (Grados)',
    'OZONO Envea': 'Concentración de ozono medida con equipo Envea (ppb)',
    'BBP': 'Porcentaje de retrodispersión de partículas (%)',
    'BC ug/m3': 'Concentración de carbono negro en el aire (µg/m3)',
    'Tempe Inter': 'Temperatura interior (°C)',
    'Humedad Inter': 'Humedad relativa interior (%)',
    'SO2 Envea': 'Concentración de dióxido de azufre medida con equipo Envea (ppb)',
    'Temp_4m': 'Temperatura a 4 metros de altura (°C)',
    'UV-BC': 'Concentración de carbono negro ultravioleta (ng/m3)',
    'HR_2m': 'Humedad relativa a 2 metros de altura (%)'
}

# Definir los datos
data = [
    ['PM10', 'µg/m3'],
    ['Vel Viento 10M', 'm/s'],
    ['Dir Viento 10M', 'Grados'],
    ['Temperatura', '°C'],
    ['Temperatura 8M', '°C'],
    ['Temperatura 20M', '°C'],
    ['HR', '%'],
    ['Presion Baro', 'mmHg'],
    ['Rad Solar', 'W/M²'],
    ['Precipitacion', 'mm'],
    ['OZONO', 'ppb'],
    ['NO', 'ppb'],
    ['NO2', 'ppb'],
    ['NOX', 'ppb'],
    ['PM2.5', 'µg/m3'],
    ['CO', 'ppm'],
    ['SO2', 'ppb'],
    ['Vel Viento', 'm/s'],
    ['Dir Viento', 'Grados'],
    ['OZONO Envea', 'ppb'],
    ['BBP', '%'],
    ['BC ug/m3', 'µg/m3'],
    ['Tempe Inter', '°C'],
    ['Humedad Inter', '%'],
    ['SO2 Envea', 'ppb'],
    ['Temp_4m', '°C'],
    ['UV-BC', 'ng/m3'],
    ['HR_2m', '%']
]

# Convertir a DataFrame
df = pd.DataFrame(data, columns=['Parámetro', 'Unidad de medida'])

# Agregar la columna de descripciones
df['Descripción'] = df['Parámetro'].map(descriptions)

df.to_excel("../Datos/RMCAB/Descripciones_Parametros_RMCAB.xlsx", index=False)

### Datos de sección cruzada

> Esta estructura permite realizar la limpieza de los datos de cada variable por estación de forma más fácil. Una vez hecho esto con esta estructura se debe pasar esta estructura a datos panel para que estos queden de forma correcta

In [35]:
cross = panel_raw.pivot_table(index=["Fecha", "Hora","Estación"], columns=["parámetros"], values="Valor")
cross = cross.reset_index()

### Limpieza ejemplo para una sola estación

In [ ]:
ex = cross[cross["Estación"] == "Centro de Alto Rendimiento"]
ex

# 

------

In [ ]:
df_pivoted = df_melted.pivot_table(index=["Fecha", "Hora","Estación"], columns=["parámetros"], values="Valor")
df_pivoted = df_pivoted.reset_index()
df_pivoted["Hora"] = (df_pivoted["Hora"]+':00:00')
df_pivoted["Hora"] = df_pivoted["Hora"].replace("24:00:00", "00:00:00")
df_pivoted["Fecha_Hora"] = pd.to_datetime(df_pivoted["Fecha"].astype(str) + " " + df_pivoted["Hora"], format="%Y-%m-%d %H:%M:%S")
df_pivoted = df_pivoted.drop(columns=["Fecha", "Hora"])

In [4]:
ex = ex.drop(columns=["Unnamed: 0","Máx","Mín","Unidades"])

df_melted = pd.melt(ex, id_vars=['parámetros', 'Fecha', 'Estación'], 
                    var_name='Hora', value_name='Valor')
df_melted["Valor"] = df_melted["Valor"].str.replace(",",".").astype(float)
df_pivoted = df_melted.pivot_table(index=["Fecha", "Hora","Estación"], columns=["parámetros"], values="Valor")
df_pivoted = df_pivoted.reset_index()
df_pivoted["Hora"] = (df_pivoted["Hora"]+':00:00')
df_pivoted["Hora"] = df_pivoted["Hora"].replace("24:00:00", "00:00:00")
df_pivoted["Fecha_Hora"] = pd.to_datetime(df_pivoted["Fecha"].astype(str) + " " + df_pivoted["Hora"], format="%Y-%m-%d %H:%M:%S")
df_pivoted = df_pivoted.drop(columns=["Fecha", "Hora"])

In [7]:
df_pivoted.loc[df_pivoted["Estación"]=="Centro de Alto Rendimiento"]

parámetros,Estación,BBP,BC ug/m3,CO,Dir Viento,Dir Viento 10M,HR,HR_2m,Humedad Inter,NO,...,SO2 Envea,Temp_4m,Tempe Inter,Temperatura,Temperatura 20M,Temperatura 8M,UV-BC,Vel Viento,Vel Viento 10M,Fecha_Hora
2,Centro de Alto Rendimiento,NaN,NaN,0.2,182.0,NaN,85.81,NaN,NaN,2.0,...,1.6,NaN,NaN,10.9,NaN,NaN,NaN,0.1,NaN,2023-01-02 01:00:00
21,Centro de Alto Rendimiento,NaN,NaN,0.3,334.0,NaN,86.88,NaN,NaN,2.3,...,1.8,NaN,NaN,10.6,NaN,NaN,NaN,0.2,NaN,2023-01-02 02:00:00
40,Centro de Alto Rendimiento,NaN,NaN,0.3,216.0,NaN,87.11,NaN,NaN,1.9,...,1.8,NaN,NaN,11.1,NaN,NaN,NaN,0.2,NaN,2023-01-02 03:00:00
59,Centro de Alto Rendimiento,NaN,NaN,0.2,74.0,NaN,86.97,NaN,NaN,2.0,...,1.8,NaN,NaN,11.4,NaN,NaN,NaN,0.4,NaN,2023-01-02 04:00:00
78,Centro de Alto Rendimiento,NaN,NaN,0.1,356.0,NaN,85.73,NaN,NaN,1.6,...,1.8,NaN,NaN,11.4,NaN,NaN,NaN,0.1,NaN,2023-01-02 05:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281974,Centro de Alto Rendimiento,9.7,0.7,0.3,154.0,NaN,55.78,NaN,NaN,0.6,...,NaN,NaN,NaN,13.7,NaN,NaN,NaN,0.8,NaN,2024-09-13 20:00:00
281993,Centro de Alto Rendimiento,9.9,0.8,0.3,81.0,NaN,59.61,NaN,NaN,0.4,...,NaN,NaN,NaN,13.0,NaN,NaN,NaN,0.3,NaN,2024-09-13 21:00:00
282012,Centro de Alto Rendimiento,11.7,1.1,0.4,360.0,NaN,66.11,NaN,NaN,0.8,...,NaN,NaN,NaN,12.0,NaN,NaN,NaN,0.0,NaN,2024-09-13 22:00:00
282031,Centro de Alto Rendimiento,12.4,1.0,0.5,351.0,NaN,66.18,NaN,NaN,4.3,...,NaN,NaN,NaN,11.3,NaN,NaN,NaN,0.6,NaN,2024-09-13 23:00:00


In [10]:
df_model = df_pivoted.pivot_table(index=['Fecha_Hora'], columns='Estación', aggfunc='first')
# Aplanar el MultiIndex en las columnas para facilitar el acceso a las columnas
df_model.columns = [' - '.join(col).strip() for col in df_model.columns.values]
# Fill NA values with the average of each column
#df_model = df_model.apply(lambda x: x.fillna(x.mean()), axis=0)
#df_model.to_excel("./Datos/RMCAB/RMCAB_to_modeling.xlsx", index=True)

In [ ]:
# Extraer el año de la columna "Fecha_Hora"
df_pivoted['Año'] = df_pivoted['Fecha_Hora'].dt.year

# Obtener las estaciones únicas por año
estaciones_por_año = df_pivoted.groupby('Año')['Estación'].unique().reset_index()

# Mostrar el resultado
for i in range(len(estaciones_por_año)):
    print("================================")
    print(f"Año {estaciones_por_año['Año'][i]}: {estaciones_por_año['Estación'][i]}, Total: {len(estaciones_por_año['Estación'][i])}")

In [ ]:
not_null_values_percentage = 100 - (df_pivoted.isnull().sum() / len(df_pivoted) * 100)
print(not_null_values_percentage)

In [ ]:
#Df_filtered = df_pivoted[df_pivoted["Fecha_Hora"].dt.year == 2024]

#df_filtered = df_pivoted.copy()
#
#for estacion in df_filtered["Estación"].unique():
#
#    df_plot = df_filtered[df_filtered["Estación"] == estacion]
#    df_plot = df_plot.set_index("Fecha_Hora")
#    df_plot = df_plot.drop(columns=["Estación"])
#    df_plot = df_plot.resample("YE").mean()
#    df_plot["Estación"] = estacion
#
#    plt.figure(figsize=(12,5))
#    plt.plot(df_plot.index, df_plot["Temperatura"])
#    plt.title(f"Temperatura en {estacion}")
